# Ep 2: MoveIt! First Steps

## Get a URDF and create a Moveit! Package

In [ ]:
roscd open_manipulator_support_description/urdf
ls open_manipulator_support.urdf.xacro

Lets have a look at the Files in the IDE -->

Now lets open the **Moveit!Wizard**:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
cd ~/catkin_ws/src
roslaunch moveit_setup_assistant setup_assistant.launch

Open the Graphical tools:

<img src="images/graphtools1.png"/>

If the windows appear decentered click this button

<img src="images/graphtools2.png"/>

We should get something like this:

<img src="images/moveitwizard1.png"/>

<img src="images/moveitwizard2.png"/>

<img src="images/moveitwizard3.png"/>

<img src="images/moveitwizard4.png"/>

Hover over the robot sim panel and zoom in with the scroll wheel ( or pinch in MAC)

<img src="images/moveitwizard5.png"/>

## Generate Collision Matrix:

The first thing that is vital for a robotarm when it moves , is to **NOT HIT ITSELF**. Which seems dumb but, its really common way to break a throusand euros robot arm if it doesnt have the correct safte features like peack torke detection or some kind of external perception.

We have to generate what is called :**Self-Collision Matrix**. To do it we use the **Self-Collision Matrix Generator** We need to generate this matrix because:
* Doing this we detect which **links will collide with each other** when moving.
* We will detect also the **links which will never collide with each other**, which then we can **remove the autocollisions calculations** for them , lowering the burden in the processing.
* Detect which **links will be always in collision** and therefore we supose that its normal and therefore we also disable the calculations.
* Disable also the **links adjacent in the kinematic chain** which obviously we will disable also their autocollisions. 

 Value is **10,000 collision means how many random robot positions to check for self collision**. The Higher the better collision detection matrix it generates, but we will need more time and processing power in paralel.

Min Collisions of 95% means that for considering that the pair is **always colliding** has to be 95% of all the random positions tested.

<img src="images/autocollision_matrix1.png"/>

<img src="images/autocollision_matrix4.png"/>

<img src="images/autocollision_matrix2.png"/>

<img src="images/autocollision_matrix3.png"/>

As you can see, in our case, 10.000 gives the same result as 100.000, therefore 10.000 was enough. In more complex robotic arms or other systems, the more links and joints , the higher number it has to be to test a bigger range of movements.

Note that because this is random, it could be that everytime you execute this, the matrix apperas a bit different:

<img src="images/10000_collisionmatrix.png"/>

<img src="images/10000_collisionsmatrix_v2.png"/>

Here Link7 with Link1 appear sometimes colliding and sometimes not. This depends on the random positions that were tested. **If you doubt, get the most retrictive version.**

<img src="images/10000_collisionsmatrix_final.png"/>

## Virtual Joints

<img src="images/virtualjoints1.png"/>

<img src="images/virtualjoints2.png"/>

A robot that moves around a plane is specified using a planar virtual joint that connnects the world frame to the frame of the robot.
A robot that doesnt move,will use a fixed joint to connect the base frame to the world.

In our case we select:
* Name: virtual_joint ( just to know its a virtual joint )
* Parent: world
* Child: the base_link , that we want to connect to world.
* Joint Type: Fixed, because we wont move.

You can play with this because, what if we select as parent a link of another robot, like a turtlebot that **moves around**? Thes ethings we will go deepre when the time comes.

** DONT FORGET TO HIT SAVE**

### Planning Groups

Doing Inverese Kinematics is computationaly very intensive. This menas that the simpler the kinematics to solve the better.
Thats why normally we divide a robot in the maximum parts that allow a correct and easy IK calculations.
Ex: a robot with TWO arms, normally will be divided into LEFT_ARM and RIGHT_ARM, because we dont need to solve inverse kinematics for both, but it all depends on the use case.

We will choose the following:
* Solver: kdl_kinematics_plugin/KDLKinematicsPlugin as the kinematics solver. This is the plugin in charge of calculate the inverse kinematics. Its a generic Kinematic solver, that will be ok for now. Its the default one in Moveit!. It only works with Serial Kinematics Chains ( tensefrities and things like that wont work).

* Name: **openmanipulator_arm** seems appropriate.
* Kin Solv. Attempts: 3 seems reasonable.
* Planner: for now we leave this none
* **We add the joins!**

In our case we select **ALL the Joints except the gripper**.

<img src="images/kinematicsolver1.png"/>

<img src="images/group_v2.png"/>

<img src="images/group_v2_2.png"/>

Now we add the gripper through joints also:
* joint8
* joint9
* id_6

NOTE that we are NOT adding anywhere the **LINK7** thats because it serves no purpose in the planning for the moment.

Other Kinematic Solvers:
If you want to create a custom Kinematic Solver for your Robotics ARM: http://docs.ros.org/kinetic/api/moveit_tutorials/html/doc/ikfast/ikfast_tutorial.html

Solver that in theory produces more reliable solutions than Jacobian methods in KDL:
http://docs.ros.org/kinetic/api/moveit_tutorials/html/doc/trac_ik/trac_ik_tutorial.html


## Generate Premade Robot Poses

We now can store premade robot poses to set the robot in safe positions, calibration, very used positions, etc.

<img src="images/finalpose1.png"/>

<img src="images/finalpose2.png"/>

<img src="images/finalpose3.png"/>

<img src="images/finalpose5.png"/>

We added four poses:
* Two for the OpenManipulator_ARM group
* Two for the Gripper group.

## End Effector:

We can add now the gripper as end effector. This unblocks some functionality related exclusively with end effectors:

We set:
* Name: gripper
* Group ENDEffector: gripper
* Parent Link: Link5
* Parent Group: Not necessary here to state it.

<img src="images/endeffector.png"/>

## Things we wont set: 

These are elements that we might enable after, but for now we leave them unset because we dont need them:
* Passive Joints: These rae for caster wheelas and other arent actuated.
* 3D perception: We dont have for the moment any sensor **( We might add it afterwards )**.
* Simulation: When executed it tels us there is nothing to change.

## ROS controllers:

Here we have two options:
* Add the controllers manually: This allows us to select which type of control we are adding.
* Auto add FollowTrajectoryControll

For the moment we will **AutoGenerate** the **FollowJointTrajectory Control**. If we need to change it we just have to **Redit the Moveit package**.

<img src="images/roscontrollers_1.png"/>

## Author: Add the Author info

<img src="images/author.png"/>

## FINALY: Generate the Package:

Now is the time to generate the package that we will use:

* Select the location. Hit browse and create a new folder with the name of the package you want. In our case we will call it **openmanipulator_ep2_movit_config**.

# TEST what we have created:

In [ ]:
cd ~/catkin_ws
source devel/setup.bash;rospack profile
roslaunch openmanipulator_ep2_movit_config demo.launch rviz_tutorial:=true

# APPENDIX

Kinematics Configurator:
http://docs.ros.org/kinetic/api/moveit_tutorials/html/doc/kinematics_configuration/kinematics_configuration_tutorial.html

Planners:
http://docs.ros.org/kinetic/api/moveit_tutorials/html/doc/ompl_interface/ompl_interface_tutorial.html
http://docs.ros.org/kinetic/api/moveit_tutorials/html/doc/chomp_planner/chomp_planner_tutorial.html
http://docs.ros.org/kinetic/api/moveit_tutorials/html/doc/stomp_planner/stomp_planner_tutorial.html

Controllers Type:
http://docs.ros.org/indigo/api/moveit_tutorials/html/doc/pr2_tutorials/planning/src/doc/controller_configuration.html#followjointtrajectory-controller-interface